**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [ARマーカが張り付けられたボックスを生成](#toc5_)    
- [光源位置の変更](#toc6_)    
- [使用関数の定義](#toc7_)    
- [カメラのパラメータ設定](#toc8_)    
- [ARマーカに関するパラメータの設定](#toc9_)    
- [ロボットアームのパラメータなどの設定](#toc10_)    
- [ビジュアルサーボの実行](#toc11_)    
- [（おまけ）特徴点位置の描画](#toc12_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは6軸のロボットアームを用いてimage based visual servoを行う方法を紹介します。

ビジュアルサーボ参考：https://github.com/RiddhimanRaut/Ur5_Visual_Servoing/blob/master/ur5_control_nodes/src/vs_ur5.py


（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
# pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定（今回は、簡単のため重力は考慮しないこととする）
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)

In [3]:
# ロボットの読み込み
armStartPos = [0, 0, 0]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple6d_arm_with_force_sensor.urdf",armStartPos, armStartOrientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
cameraDistance = 1.5
cameraYaw = 50.0 # deg
cameraPitch = -40 # deg
cameraTargetPosition = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[ARマーカが張り付けられたボックスを生成](#toc0_)

In [4]:
# 一面にARマーカが描かれたボックスの生成
box_bid = pybullet.loadURDF("../urdf/ar_marker_box.urdf", [0, 0, 0.5], useFixedBase=True)

# textureを設定(urdfファイルで指定したものと、同じものを指定)
texture_id = pybullet.loadTexture("../texture/ar_marker_box.png")
pybullet.changeVisualShape(box_bid, -1, textureUniqueId=texture_id)

# ARマーカが張り付けられたボックスの位置を調整するスライダーを設定 
pybullet.addUserDebugParameter("obj_x", -4, 4, 0.0)
pybullet.addUserDebugParameter("obj_y", -2, 0, -1.5)
pybullet.addUserDebugParameter("obj_z", -4, 8, 1.3)
pybullet.addUserDebugParameter("obj_roll", -3.14, 3.14, 0.0)
pybullet.addUserDebugParameter("obj_pitch", -3.14, 3.14, 0)
pybullet.addUserDebugParameter("obj_yaw", -3.14, 3.14, 0)

5

# <a id='toc6_'></a>[光源位置の変更](#toc0_)

In [5]:
# デフォルトの光源位置だと、ARマーカが認識されにくいので、光源位置を変更
pybullet.configureDebugVisualizer(lightPosition=[0, 0, 10])

# <a id='toc7_'></a>[使用関数の定義](#toc0_)

In [6]:
import numpy as np
import cv2

def Rx(theta):
    """
    x軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸周りの回転行列
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    y軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸周りの回転行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    z軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸周りの回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])


def Hx(theta):
    """
    x軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸回りの同次変換行列
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    y軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    z軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    平行移動の同次変換行列を求める

    Parameters
    ----------
    x : float
        x方向の移動量
    y : float
        y方向の移動量
    z : float
        z方向の移動量

    Returns
    -------
    numpy.ndarray
        平行移動の同次変換行列
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])
                     
def Make6dJacobianMatrix(Q, o1, o2, o3, o4, o5, o6, oc):
    """
    6軸ロボットアームの基礎ヤコビ行列を求める

    Parameters
    ----------
    Q : numpy.ndarray
        関節角度
    o1 : numpy.ndarray
        リンク1の原点
    o2 : numpy.ndarray
        リンク2の原点
    o3 : numpy.ndarray
        リンク3の原点
    o4 : numpy.ndarray
        リンク4の原点
    o5 : numpy.ndarray
        リンク5の原点
    o6 : numpy.ndarray
        リンク6の原点
    oc : numpy.ndarray
        カメラの原点
   
        
    Returns
    -------
    Jv : numpy.ndarray
         基礎ヤコビ行列
    """

    
    R12 = Hz(theta=Q[0])
    R23 = Hy(theta=Q[1])
    R34 = Hy(theta=Q[2])
    R45 = Hz(theta=Q[3])
    R56 = Hy(theta=Q[4])
    R6C = Hz(theta=Q[5] - np.pi/2)
    ex = np.array([1, 0, 0, 1])
    ey = np.array([0, 1, 0, 1])
    ez = np.array([0, 0, 1, 0])

    # ヤコビ行列の各要素を計算
    a1 = (R12 @ ez)[0:3]
    a2 = (R12 @ R23 @ ey)[0:3]
    a3 = (R12 @ R23 @ R34 @ ey)[0:3]
    a4 = (R12 @ R23 @ R34 @ R45 @ ez)[0:3]
    a5 = (R12 @ R23 @ R34 @ R45 @ R56 @ ey)[0:3]
    a6 = (R12 @ R23 @ R34 @ R45 @ R56 @ R6C @ ez)[0:3]

    # Ji = [[ai x (oc - oi)], 
    #       [ai]]
    j1 = np.concatenate((np.cross(a1, oc-o1).reshape(3, 1), a1.reshape(3, 1)), axis=0)
    j2 = np.concatenate((np.cross(a2, oc-o2).reshape(3, 1), a2.reshape(3, 1)), axis=0)
    j3 = np.concatenate((np.cross(a3, oc-o3).reshape(3, 1), a3.reshape(3, 1)), axis=0)
    j4 = np.concatenate((np.cross(a4, oc-o4).reshape(3, 1), a4.reshape(3, 1)), axis=0)
    j5 = np.concatenate((np.cross(a5, oc-o5).reshape(3, 1), a5.reshape(3, 1)), axis=0)
    j6 = np.concatenate((np.cross(a6, oc-o6).reshape(3, 1), a6.reshape(3, 1)), axis=0)

    Jv = np.concatenate((j1, j2, j3, j4, j5, j6), axis=1)
    return Jv

def MakeImageJacobianMatrix(current_feature_points, camera_to_obj, f):
    """
    画像ヤコビ行列を求める

    Parameters
    ----------
    current_feature_points : numpy.ndarray
        現在の特徴点座標
    camera_to_objs : float
        カメラからオブジェクトまでの距離
    f : float
        カメラの焦点距離
    
    Returns
    -------
    Ji : numpy.ndarray
        画像ヤコビ行列
    """
    # 4つの特徴点の画像ヤコビ行列
    u1 = current_feature_points[0]
    v1 = current_feature_points[1]
    u2 = current_feature_points[2]
    v2 = current_feature_points[3]
    u3 = current_feature_points[4]
    v3 = current_feature_points[5]
    u4 = current_feature_points[6]
    v4 = current_feature_points[7]

    Ji = np.array([[-f*(1/camera_to_obj), 0, u1*(1/camera_to_obj), u1*v1/f, -(f+(1/f)*u1**2), v1],
                   [0, -f*(1/camera_to_obj), v1*(1/camera_to_obj), f+(1/f)*v1**2, -(1/f)*u1*v1, -u1],
                   [-f*(1/camera_to_obj), 0, u2*(1/camera_to_obj), u2*v2/f, -(f+(1/f)*u2**2), v2],
                   [0, -f*(1/camera_to_obj), v2*(1/camera_to_obj), f+(1/f)*v2**2, -(1/f)*u2*v2, -u2],
                   [-f*(1/camera_to_obj), 0, u3*(1/camera_to_obj), u3*v3/f, -(f+(1/f)*u3**2), v3],
                   [0, -f*(1/camera_to_obj), v3*(1/camera_to_obj), f+(1/f)*v3**2, -(1/f)*u3*v3, -u3],
                   [-f*(1/camera_to_obj), 0, u4*(1/camera_to_obj), u4*v4/f, -(f+(1/f)*u4**2), v4],
                   [0, -f*(1/camera_to_obj), v4*(1/camera_to_obj), f+(1/f)*v4**2, -(1/f)*u4*v4, -u4]])
    return Ji

def DetectArMarkerCornerPosAndDepth(markerSize, arucoDict, parameters, rgbImg, cameraMatrix, distCoeffs):
    """
    ARマーカの4隅の位置とカメラまでの距離を検出する

    Parameters
    ----------
    markerSize : float
        ARマーカの1辺の長さ（メートル）
    arucoDict : cv2.aruco.Dictionary
        ARマーカの辞書
    parameters : cv2.aruco.DetectorParameters
        ARマーカ検出のパラメータ
    rgbImg : numpy.ndarray
        カメラ画像（RGB）
    cameraMatrix : numpy.ndarray
        カメラの内部パラメータ行列
    distCoeffs : numpy.ndarray
        歪み係数

    Returns
    -------
    corner_pos : numpy.ndarray
        ARマーカの4つの角の位置(x1, y1, x2, y2, x3, y3, x4, y4)[pixel位置]
    depth : float
        カメラからARマーカまでの距離[m]
    """

    # カメラ画像をグレースケールに変換
    rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_RGB2BGR)

    # ARマーカの検出
    corners, ids, _ = cv2.aruco.detectMarkers(rgbImg, arucoDict, parameters=parameters)

    # 検出されたマーカがない場合はNoneを返す
    if ids is None or len(ids) == 0:
        return None, None, None

    # 最初に検出されたマーカの4隅の位置を取得
    marker_corners = corners[0][0]

    # 各隅の位置と対応する深度を取得
    cornerPos = np.zeros(8)
    for i in range(4):
        x, y = int(marker_corners[i][0]), int(marker_corners[i][1])
        # depth = depthImg[y, x]
        cornerPos[i*2] = x
        cornerPos[i*2+1] = y

    # aolvePnPでカメラからARマーカまでの距離を求める
    # 3次元座標
    markerSize = 0.1
    objp = np.array([[-markerSize/2, -markerSize/2, 0],
                     [markerSize/2, -markerSize/2, 0],
                     [markerSize/2, markerSize/2, 0],
                     [-markerSize/2, markerSize/2, 0]], dtype=np.float32)
    # 2次元座標
    imagePoints = np.array([[cornerPos[0], cornerPos[1]],
                             [cornerPos[2], cornerPos[3]],
                             [cornerPos[4], cornerPos[5]],
                             [cornerPos[6], cornerPos[7]],], dtype=np.float32)

    # solvePnPでカメラからARマーカまでの距離を求める
    _, _, translation_vector = cv2.solvePnP(objp, imagePoints, cameraMatrix, distCoeffs)

    # カメラからARマーカまでの距離を求める
    depth = translation_vector[2][0]

    return cornerPos, depth



# <a id='toc8_'></a>[カメラのパラメータ設定](#toc0_)

In [7]:
# カメラ設定
fov = 60
imageWidth = 224
imageHeight = 224
centerX = imageWidth / 2
centerY = imageHeight / 2
aspect = imageWidth / imageHeight
near = 0.05
far = 10
projectionMatrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# 焦点距離を求める
fovRad = np.deg2rad(fov)
f = imageHeight / (2 * np.tan(fovRad / 2))

# カメラの内部パラメータ
cameraMatrix = np.array([[f, 0, imageWidth/2],
                         [0, f, imageHeight/2],
                         [0, 0, 1]], dtype=np.float32)

# 歪み係数（ここでは、歪みがないと仮定）
distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc9_'></a>[ARマーカに関するパラメータの設定](#toc0_)

In [8]:
markerSize = 0.1 # ARマーカの1辺の長さ（メートル）

# 検出するARマーカの辞書を定義
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

# <a id='toc10_'></a>[ロボットアームのパラメータなどの設定](#toc0_)

In [9]:
# 各リンクの長さ
worldToArmOriginX = 0.0 # ワールド座標系から見たアームの原点のx座標
worldToArmOriginY = 0.0 # ワールド座標系から見たアームの原点のy座標
worldToArmOriginZ = 0.8 # ワールド座標系から見たアームの原点のz座標(「simple6d_arm_with_gripper.urdf」の base_link の z方向の長さ)
link1Length = 0.3 # link1の長さ(「simple6d_arm_with_gripper.urdf」の link1 の z方向の長さ)
link2Length = 0.5 # link2の長さ(「simple6d_arm_with_gripper.urdf」の link2 の z方向の長さ)
link3Length = 0.5 # link3の長さ(「simple6d_arm_with_gripper.urdf」の link3 の z方向の長さ)
link4Length = 0.1 # link4の長さ(「simple6d_arm_with_gripper.urdf」の link4 の z方向の長さ)
link5Length = 0.1 # link5の長さ(「simple6d_arm_with_gripper.urdf」の link5 の z方向の長さ)
link6Length = 0.15 # link6の長さ(「simple6d_arm_with_gripper.urdf」の link6 の z方向の長さ)
cameraZSize = 0.01 # カメラのz方向のサイズ
cameraXOffset = 0.08 # link6座標系の原点からカメラ座標系の原点までのx方向のオフセット

# ここの値を変えると、結果が変わります#############
# 各リンクの初期角度を定義
link1AngleDeg = -90
link2AngleDeg = 55
link3AngleDeg = 35
link4AngleDeg = 0
link5AngleDeg = 0
link6AngleDeg = 0
featureParam = 10

# 画像上の4つの特徴量の目標位置
goal1U = centerX - 50
goal1V = centerY - 50
goal2U = centerX + 50
goal2V = centerY - 50
goal3U = centerX + 50
goal3V = centerY + 50
goal4U = centerX - 50
goal4V = centerY + 50
##############################################

# deg -> radへ変換
link1AngleRad  = np.deg2rad(link1AngleDeg)
link2AngleRad  = np.deg2rad(link2AngleDeg)
link3AngleRad  = np.deg2rad(link3AngleDeg)
link4AngleRad  = np.deg2rad(link4AngleDeg)
link5AngleRad  = np.deg2rad(link5AngleDeg)
link6AngleRad  = np.deg2rad(link6AngleDeg)

# 「4つの特徴量の目標位置」をリストにまとめる
goalFeaturePoints = np.array([goal1U, goal1V, goal2U, goal2V, goal3U, goal3V, goal4U, goal4V])

# ロボットの各関節のインデックス
link1JointIdx = 0
link2JointIdx = 1
link3JointIdx = 2
link4JointIdx = 3
link5JointIdx = 4
link6JointIdx = 5
cameraIdx = 6
cameraTargetIdx = 7

# ロボットの各関節の角度を初期化
pybullet.resetJointState(armId, link1JointIdx, link1AngleRad )
pybullet.resetJointState(armId, link2JointIdx, link2AngleRad )
pybullet.resetJointState(armId, link3JointIdx, link3AngleRad )
pybullet.resetJointState(armId, link4JointIdx, link4AngleRad )
pybullet.resetJointState(armId, link5JointIdx, link5AngleRad )
pybullet.resetJointState(armId, link6JointIdx, link6AngleRad )

# <a id='toc11_'></a>[ビジュアルサーボの実行](#toc0_)

右側のスライダーの値を変更することで、ARマーカを貼り付けたボックスの位置・姿勢を変更することができます。

なおデフォルトのロボットの姿勢、ボックスの位置の場合「x, roll, yaw」方向の変更は上手く調整しないと適切に動作しないことがあります。  
そのため、最初は「y, z, pitch」方向の変更をお勧めします。

In [10]:
import threading
import time

# グローバル変数（おまけの章でも使用する変数をglobal変数として定義）
currentFeaturePoints = None # 特徴点の現在位置
rgbImg = None # カメラから取得したRGB画像

def VisualServo():
    global currentFeaturePoints
    global rgbImg

    while True:
        # 「ARマーカが張り付けられたボックス」をスライダーに設定された位置、姿勢に設定
        objX = pybullet.readUserDebugParameter(0)
        objY = pybullet.readUserDebugParameter(1)
        objZ = pybullet.readUserDebugParameter(2)
        objRoll = pybullet.readUserDebugParameter(3)
        objPitch = pybullet.readUserDebugParameter(4)
        objYaw = pybullet.readUserDebugParameter(5)
        pybullet.resetBasePositionAndOrientation(box_bid, [objX, objY, objZ], pybullet.getQuaternionFromEuler([objRoll, objPitch, objYaw]))

        # 関節角度を取得
        q1 = pybullet.getJointState(armId, link1JointIdx)[0]
        q2 = pybullet.getJointState(armId, link2JointIdx)[0]
        q3 = pybullet.getJointState(armId, link3JointIdx)[0]
        q4 = pybullet.getJointState(armId, link4JointIdx)[0]
        q5 = pybullet.getJointState(armId, link5JointIdx)[0]
        q6 = pybullet.getJointState(armId, link6JointIdx)[0]

        # ロボットアームの手先のカメラの姿勢に合わせて、カメラの上方向のベクトルを設定
        cameraUpVector = np.array([0, -1, 0]) # デフォルトのカメラの上方向のベクトル
        RW1 = Rz(theta=0)@Ry(theta=0)@Rx(theta=0) # ワールド座標系 -> link1座標系
        R12 = Rz(theta=q1) # link1座標系 -> link2座標系
        R23 = Ry(theta=q2) # link2座標系 -> link3座標系
        R34 = Ry(theta=q3) # link3座標系 -> link4座標系
        R45 = Rz(theta=q4) # link4座標系 -> link5座標系
        R56 = Ry(theta=q5) # link5座標系 -> link6座標系
        R6C = Rz(theta=q6-np.pi/2) # link6座標系 -> カメラ座標系（カメラを-90度回転すると、取得される画像が正しい向きになる）
        RWC = RW1@R12@R23@R34@R45@R56@R6C # ワールド座標系 -> カメラ座標系
        rotateCameraUpVector = RWC@cameraUpVector # カメラ座標系におけるカメラの上方向のベクトル

        # カメラ画像を取得
        cameraLinkPose = pybullet.getLinkState(armId, cameraIdx)[0] # 手先のカメラリンクの位置
        cameraTargetLinkPose = pybullet.getLinkState(armId, cameraTargetIdx)[0] # カメラリンクの少しだけ前に設定した仮想的なリンクの位置
        viewMatrix = pybullet.computeViewMatrix(cameraEyePosition=[cameraLinkPose[0], cameraLinkPose[1], cameraLinkPose[2]],cameraTargetPosition=[cameraTargetLinkPose[0], cameraTargetLinkPose[1], cameraTargetLinkPose[2]],cameraUpVector=rotateCameraUpVector)
        _, _, rgbImg, _, _ = pybullet.getCameraImage(
            width=imageWidth,
            height=imageHeight,
            viewMatrix=viewMatrix,
            projectionMatrix=projectionMatrix,
            renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
        )

        # 「画像座標系における4つの特徴量の位置（ピクセル位置）」、「カメラ→オブジェクトの距離[m]」を取得
        currentFeaturePoints, z = DetectArMarkerCornerPosAndDepth(markerSize, arucoDict, parameters, rgbImg, cameraMatrix, distCoeffs)

        # 特徴量が検出できなかった場合は、次のループへ
        if currentFeaturePoints is None:
            continue

        # 画像ヤコビ行列
        Ji = MakeImageJacobianMatrix(currentFeaturePoints, z, f)
        JiInv = np.linalg.pinv(Ji) # 画像ヤコビ行列の逆行列

        # ワールド座標系から見た、各linkの座標系の位置を計算（基礎ヤコビ行列の計算に使用）
        TW1 = Hz(theta=0)@Hy(theta=0)@Hx(theta=0)@Hp(x=worldToArmOriginX, y=worldToArmOriginY, z=worldToArmOriginZ) # ワールド座標系 -> link1座標系
        T12 = Hz(theta=q1)@Hp(x=0, y=0, z=link1Length) # リンク1座標系 -> リンク2座標系
        T23 = Hy(theta=q2)@Hp(x=0, y=0, z=link2Length) # リンク2座標系 -> リンク3座標系
        T34 = Hy(theta=q3)@Hp(x=0, y=0, z=link3Length) # リンク3座標系 -> リンク4座標系
        T45 = Hz(theta=q4)@Hp(x=0, y=0, z=link4Length) # リンク4座標系 -> リンク5座標系
        T56 = Hy(theta=q5)@Hp(x=0, y=0, z=link5Length) # リンク5座標系 -> リンク6座標系
        T6C = Hz(theta=q6-np.pi/2)@Hp(x=cameraXOffset, y=0.0, z=link6Length-cameraZSize/2) # link6座標系 -> カメラ座標系
        originPos = np.array([0, 0, 0, 1]) # 原点位置
        originPosLink1 = TW1@originPos                         # ワールド座標系から見た、リンク1座標系の原点位置
        originPosLink2 = TW1@T12@originPos                     # ワールド座標系から見た、リンク2座標系の原点位置
        originPosLink3 = TW1@T12@T23@originPos                 # ワールド座標系から見た、リンク3座標系の原点位置
        originPosLink4 = TW1@T12@T23@T34@originPos             # ワールド座標系から見た、リンク4座標系の原点位置
        originPosLink5 = TW1@T12@T23@T34@T45@originPos         # ワールド座標系から見た、リンク5座標系の原点位置
        originPosLink6 = TW1@T12@T23@T34@T45@T56@originPos     # ワールド座標系から見た、リンク6座標系の原点位置
        originPosLinkC = TW1@T12@T23@T34@T45@T56@T6C@originPos # ワールド座標系から見た、カメラ座標系の原点位置

        # 基礎ヤコビ行列
        Jv = Make6dJacobianMatrix(Q=[q1, q2, q3, q4, q5, q6], o1=originPosLink1[0:3], o2=originPosLink2[0:3], o3=originPosLink3[0:3], o4=originPosLink4[0:3], o5=originPosLink5[0:3], o6=originPosLink6[0:3], oc=originPosLinkC[0:3])
        JvInv = np.linalg.pinv(Jv) # 基礎ヤコビ行列の逆行列

        # 「現在の特徴量の位置」→「目標の特徴量の位置」方向のベクトル
        featureCurrentToGoal = goalFeaturePoints - currentFeaturePoints

        # 「現在の特徴量の位置」→「目標の特徴量の位置」方向に向かう微小変化量
        deltaFeaturePoints = featureParam * featureCurrentToGoal

        # 「画像ヤコビ行列の逆行列Ji_inv」を用いて、「特徴点の微小変化量delta_feature_points」から、「カメラの微小変化量（=Δx, Δy, Δz, Δroll, Δpitch, Δyaw）camera_delta_p」を求める
        # （カメラをどのように動かせば、特徴点が目標の位置に移動するか）
        DeltaPCamera = JiInv @ deltaFeaturePoints

        # Rを対角に持った対角行列
        size = 6
        RotateDiagonalMatrix = np.zeros((size, size))
        for i in range(0, size, 3):
            RotateDiagonalMatrix[i:i+3, i:i+3] = RWC

        # 「カメラ座標系を基準としたカメラの微小変化量camera_delta_p」を回転行列を用いて「ワールド座標系を基準としたカメラの微小変化量robot_delta_p」に変換する
        DeltaPWorld = RotateDiagonalMatrix @ DeltaPCamera

        # 「（基礎）ヤコビ行列の逆行列Jv_inv」を用いて、「ワールド座標系を基準としたカメラの微小変化量robot_delta_p」から、「各関節の角速度robot_delta_p」を求める
        # （ロボットアームの各関節をどのように動かせば、カメラが目標の位置、姿勢に移動するか）
        deltaQ = JvInv @ DeltaPWorld

        # 求めた角速度を各関節にセット
        pybullet.setJointMotorControl2(armId, link1JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[0])
        pybullet.setJointMotorControl2(armId, link2JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[1])
        pybullet.setJointMotorControl2(armId, link3JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[2])
        pybullet.setJointMotorControl2(armId, link4JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[3])
        pybullet.setJointMotorControl2(armId, link5JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[4])
        pybullet.setJointMotorControl2(armId, link6JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=deltaQ[5])

        # 1時刻分だけ進める
        pybullet.stepSimulation()
        time.sleep(timeStep)

# スレッドを立ち上げる
thread = threading.Thread(target=VisualServo)
thread.start()

# <a id='toc12_'></a>[（おまけ）特徴点位置の描画](#toc0_)

下コードを実行すると、「ゴールの特徴点位置」と「現在の特徴点位置」が描画されます。

具体的に、特徴点がどのように移動しているのかを確認することができます。

In [11]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.ion()  # インタラクティブモードを有効にする
fig, ax = plt.subplots()
goal_plot, = ax.plot([], [], 'ro', label="Goal Points")
current_plot, = ax.plot([], [], 'bo', label="Current Points")
ax.set_xlim(0, imageWidth)
ax.set_ylim(0, imageHeight)
ax.invert_yaxis()  # 画像座標系と一致させるためにy軸を反転

plt.legend()
plt.show()

# メインループで、カメラから取得した画像を描画
initDraw = True
while True:
    # rgbImgを描画し、goal_feature_pointsとcurrent_feature_pointsを描画
    try:
        if goalFeaturePoints is not None and currentFeaturePoints is not None:
            if initDraw:
                img_plot = ax.imshow(rgbImg)
                initDraw = False
            else:
                img_plot.set_data(rgbImg)
            goal_plot.set_xdata([goal1U, goal2U, goal3U, goal4U])
            goal_plot.set_ydata([goal1V, goal2V, goal3V, goal4V])
            current_plot.set_xdata([currentFeaturePoints[0], currentFeaturePoints[2], currentFeaturePoints[4], currentFeaturePoints[6]])
            current_plot.set_ydata([currentFeaturePoints[1], currentFeaturePoints[3], currentFeaturePoints[5], currentFeaturePoints[7]])
            plt.pause(0.001)
    except:
        pass